In [10]:
!pip3 install requests
!pip3 install bs4
!pip install requests

import requests
from bs4 import BeautifulSoup

In [11]:
# get html
page = requests.get('https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&diff=945633050&oldid=942851379')
soup = BeautifulSoup(page.text, 'html.parser')

In [12]:
import pandas as pd

df = pd.read_html(str(soup.find_all('table')[1]))[0]

In [13]:
#df = pd.DataFrame(data=df)
df

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
...,...,...,...
282,M8Z,Etobicoke,Mimico NW
283,M8Z,Etobicoke,The Queensway West
284,M8Z,Etobicoke,Royal York South West
285,M8Z,Etobicoke,South of Bloor


In [14]:
# Remove unassigned postcodes
df = df[df['Borough']!='Not assigned']
df.reset_index(inplace=True)
df

,index,Postcode,Borough,Neighbourhood
0,2,M3A,North York,Parkwoods
1,3,M4A,North York,Victoria Village
2,4,M5A,Downtown Toronto,Harbourfront
3,5,M6A,North York,Lawrence Heights
4,6,M6A,North York,Lawrence Manor
...,...,...,...,...
205,281,M8Z,Etobicoke,Kingsway Park South West
206,282,M8Z,Etobicoke,Mimico NW
207,283,M8Z,Etobicoke,The Queensway West
208,284,M8Z,Etobicoke,Royal York South West


In [15]:
!pip3 install geocoder
import geocoder

pc_list = []

for i in range(0,len(df)):
#for i in range(0,10):
    address='{}, {}, {}, Toronto, Ontario'.format(df.at[i,'Postcode'],df.at[i,'Borough'],df.at[i,'Neighbourhood'])
    #df_pc['Postcode'] = df.at[i,'Postcode']
    #print(address)
    #df['Latitude'][i],df['Longitude'][i]=geocoder.arcgis(address)
    g = geocoder.arcgis(address)
    #print(g.json)
    pc_list.append([df.at[i,'Postcode'],df.at[i,'Borough'],df.at[i,'Neighbourhood'], g.json['lat'], g.json['lng']])
    #print(Lat_list.append(g.latlng[0]),Lng_list.append(g.latlng[1]))

In [16]:
# convert list to dataframe
pc_df = pd.DataFrame(data=pc_list,columns=['Postcode','Borough','Neighbourhood','Latitude','Longitude'])
pc_df.head(5)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.752420,-79.329242
1,M4A,North York,Victoria Village,43.730600,-79.313265
2,M5A,Downtown Toronto,Harbourfront,43.650295,-79.359166
3,M6A,North York,Lawrence Heights,43.723570,-79.437110
4,M6A,North York,Lawrence Manor,43.722920,-79.431310


In [17]:
# join dataframes on postcode
df = pd.merge(df,pc_df, on=['Postcode','Borough','Neighbourhood'], how='inner')
df.head(5)

,index,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,2,M3A,North York,Parkwoods,43.752420,-79.329242
1,3,M4A,North York,Victoria Village,43.730600,-79.313265
2,4,M5A,Downtown Toronto,Harbourfront,43.650295,-79.359166
3,5,M6A,North York,Lawrence Heights,43.723570,-79.437110
4,6,M6A,North York,Lawrence Manor,43.722920,-79.431310


In [18]:
df.shape

(210, 6)